# Instala bibliotecas necessárias e importa módulos essenciais.



In [1]:
!pip install -q unsloth[colab-new] faiss-cpu sentence-transformers trl transformers datasets
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install -U spacy
!python -m spacy download pt_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import json
import re
import os
import unicodedata
import faiss
import numpy as np
import torch
import random
import pandas as pd
import sqlite3
import spacy

from unsloth import FastLanguageModel, is_bfloat16_supported
from transformers import pipeline
from transformers import TrainingArguments
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
from datasets import Dataset
from trl import SFTTrainer
from dotenv import load_dotenv
from huggingface_hub import login
from datetime import date, timedelta

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Download e exploração inicial dos dados

In [4]:
!git clone https://github.com/pubmedqa/pubmedqa.git

file_path = 'pubmedqa/data/ori_pqal.json'

with open(file_path, 'r') as f:
    data = json.load(f)

sample_key = list(data.keys())[0]
print(f"\nCampos disponíveis: {list(data[sample_key].keys())}\n")

print("=" * 60)
print("Exploração de dados - PubMedQA")
print("=" * 60)

for i, key in enumerate(list(data.keys())[:3]):
    item = data[key]

    print(f"\nExemplo {i+1} | ID: {key}")
    print("-" * 60)
    print(f"Question: {item.get('QUESTION', 'N/A')}")

    context = " ".join(item.get('CONTEXTS', []))
    print(f"Context: {context[:300]}...")

    print(f"Labels: {item.get('LABELS', 'N/A')}")
    print(f"Decision: {item.get('final_decision', 'N/A')}")
    print(f"Answer: {item.get('LONG_ANSWER', 'N/A')[:200]}...")
    print(f"Meshes: {item.get('MESHES', 'N/A')}")
    print(f"Year: {item.get('YEAR', 'N/A')}")
    print(f"Reasoning required pred: {item.get('reasoning_required_pred', 'N/A')}")
    print(f"Reasoning free pred: {item.get('reasoning_free_pred', 'N/A')}")

print(f"\n\nTotal de registros: {len(data)}")

Cloning into 'pubmedqa'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 40 (delta 0), reused 2 (delta 0), pack-reused 37 (from 1)
Receiving objects: 100% (40/40), 704.89 KiB | 1.77 MiB/s, done.
Resolving deltas: 100% (12/12), done.

Campos disponíveis: ['QUESTION', 'CONTEXTS', 'LABELS', 'MESHES', 'YEAR', 'reasoning_required_pred', 'reasoning_free_pred', 'final_decision', 'LONG_ANSWER']

Exploração de dados - PubMedQA

Exemplo 1 | ID: 21645374
------------------------------------------------------------
Question: Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?
Context: Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD 

# Pré-processamento e Preparação para RAG
## - Limpeza, Normalização e Anonimização dos Textos

In [5]:
# Define a função para anonimizar dados sensíveis em um texto.
def anonymize_text(text):
    """Remove dados sensíveis (LGPD/HIPAA compliance)"""
    if not isinstance(text, str):
        return ""
    text = re.sub(r'(Dr\.|Dra\.|Doctor|Prof\.|MD)\s+[A-Z][a-z]+(\s+[A-Z][a-z]+)?', '[NOME]', text)
    text = re.sub(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', '[EMAIL]', text)
    locations = r'(Israel|Denmark|Chile|Texas|France|United Kingdom|UK|USA|Pakistan|Karachi|Jordan|Japan|Australia|North Carolina|Washington)'
    text = re.sub(locations, '[LOCAL]', text, flags=re.IGNORECASE)
    text = re.sub(r'\b\d{6,}\b', '[ID]', text)
    text = re.sub(r'\b(19|20)\d{2}\b', '[ANO]', text)
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '[URL]', text)
    return text

# Define a função para limpar e normalizar o texto, aplicando também a anonimização.
def clean_text(text):
  if not text:
    return ""
  text = unicodedata.normalize("NFKC", text)
  text = re.sub(r"\s+", " ", text).strip()
  text = anonymize_text(text)
  return text

rag_documents = []

# Processa cada item dos dados, aplicando as funções de limpeza e anonimização.
# Cria uma lista de dicionários com as informações processadas.
for item in data.values():
    question = clean_text(item.get("QUESTION"))
    context = clean_text(" ".join(item.get("CONTEXTS", [])))
    answer = clean_text(item.get("LONG_ANSWER", ""))

    if not question or not context:
        continue

    text = f"""
    Pergunta científica:
    {question}

    Evidência:
    {context}

    Conclusão:
    {answer}
    """

    rag_documents.append(text.strip())

# Exibe o número total de documentos processados.
print(len(rag_documents))

1000


# Geração de Embeddings e Construção de Índice FAISS

In [6]:
# Inicializa o modelo de embeddings para converter texto em vetores numéricos.
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Gera os embeddings para todos os documentos RAG.
embeddings = embedder.encode(rag_documents, show_progress_bar=True)

# Cria um índice FAISS para busca eficiente de documentos similares.
index = faiss.IndexFlatL2(embeddings.shape[1])

# Adiciona os embeddings ao índice FAISS.
index.add(np.array(embeddings))

# Garante que o diretório para salvar os arquivos exista no Google Drive.
os.makedirs('/content/drive/MyDrive/rag', exist_ok=True)

# Salva o índice FAISS no Google Drive.
faiss.write_index(index, "/content/drive/MyDrive/rag/medical_index.faiss")

# Salva os documentos RAG originais em formato JSON no Google Drive.
with open('/content/drive/MyDrive/rag/medical_docs.json', 'w') as f:
  json.dump(rag_documents, f)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

# Criação de dataset de prontuário (Fictício) com SQLite

In [7]:
# Define o caminho para o arquivo do banco de dados SQLite.
DB_PATH = "prontuarios.db"

# Conecta ao banco de dados SQLite e cria um cursor.
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Cria a tabela 'pacientes' se ela ainda não existir, com as colunas especificadas.
cursor.execute("""
CREATE TABLE IF NOT EXISTS pacientes (
    patient_id TEXT PRIMARY KEY,
    nome TEXT,
    data_nascimento TEXT,
    idade INTEGER,
    sexo TEXT,
    alergias TEXT,
    comorbidades TEXT
)
""")

# Cria a tabela 'atendimentos' se ela ainda não existir, com as colunas especificadas e uma chave estrangeira para 'pacientes'.
cursor.execute("""
CREATE TABLE IF NOT EXISTS atendimentos (
    atendimento_id INTEGER PRIMARY KEY AUTOINCREMENT,
    patient_id TEXT,
    data_atendimento TEXT,
    queixa_principal TEXT,
    anamnese TEXT,
    diagnostico TEXT,
    conduta TEXT,
    tratamentos_em_andamento TEXT,
    exames_solicitados TEXT,
    observacoes TEXT,
    FOREIGN KEY(patient_id) REFERENCES pacientes(patient_id)
)
""")

# Salva as mudanças no banco de dados e fecha a conexão.
conn.commit()
conn.close()

# Populando os dados fictícios

In [8]:
# Listas de dados fictícios para nomes, diagnósticos, alergias e comorbidades.
nomes = [
    "Ana Paula Souza", "Ana Carolina Lima", "Bruno Silva", "Carlos Eduardo Rocha",
    "Daniela Martins", "Eduardo Nogueira", "Fernanda Alves", "Gabriel Pacheco",
    "Helena Ribeiro", "Igor Farias", "Juliana Torres", "Lucas Fernandes",
    "Mariana Araujo", "Natalia Pacheco", "Otavio Nunes", "Paula Guedes",
    "Rafael Moreira", "Sabrina Lopes", "Thiago Barros", "Vanessa Farias",
    "William Teixeira", "Ana Beatriz Costa"
]

diagnosticos = [
    "Hipertensão arterial sistêmica",
    "Diabetes mellitus tipo 2",
    "Asma brônquica",
    "Infecção do trato urinário",
    "Pneumonia adquirida na comunidade",
    "Transtorno de ansiedade generalizada",
    "Gastrite crônica",
    "Enxaqueca crônica"
]

alergias_lista = [
    "Dipirona", "Penicilina", "Sulfa", "Nenhuma conhecida"
]

comorbidades_lista = [
    "Hipertensão", "Diabetes", "Dislipidemia", "Obesidade", "Nenhuma"
]

# Função auxiliar para gerar datas de nascimento baseadas na idade.
def gerar_data_nascimento(idade):
    hoje = date.today()
    return hoje - timedelta(days=idade * 365)

# Conecta ao banco de dados SQLite para inserção dos dados.
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Loop para gerar e inserir dados de pacientes fictícios na tabela 'pacientes'.
for i, nome in enumerate(nomes, start=1):
    idade = random.randint(18, 85)
    patient_id = f"PAT{i:04d}"

    cursor.execute("""
        INSERT OR IGNORE INTO pacientes
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (
        patient_id,
        nome,
        gerar_data_nascimento(idade).isoformat(),
        idade,
        random.choice(["F", "M"]),
        random.choice(alergias_lista),
        random.choice(comorbidades_lista)
    ))

    # Loop interno para gerar e inserir múltiplos atendimentos para cada paciente na tabela 'atendimentos'.
    for _ in range(random.randint(1, 4)):
        cursor.execute("""
            INSERT INTO atendimentos (
                patient_id,
                data_atendimento,
                queixa_principal,
                anamnese,
                diagnostico,
                conduta,
                tratamentos_em_andamento,
                exames_solicitados,
                observacoes
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            patient_id,
            (date.today() - timedelta(days=random.randint(1, 1200))).isoformat(),
            "Dor, mal-estar e sintomas gerais",
            "Paciente relata início dos sintomas há alguns dias, sem fatores agravantes claros.",
            random.choice(diagnosticos),
            "Conduta expectante e acompanhamento ambulatorial",
            "Uso contínuo de medicação conforme prescrição",
            "Hemograma completo, glicemia, PCR",
            "Paciente orientado quanto aos sinais de alarme"
        ))

# Salva as mudanças no banco de dados e fecha a conexão.
conn.commit()
conn.close()

# Preparando conjunto de dados em português para treino de tradução

In [9]:
# Clona o repositório contendo o dataset para alinhamento de linguagem em português.
!git clone https://github.com/diegosdomingos/tech-challenge-3.git

# Carrega o dataset a partir do arquivo JSONL.
file_path = 'tech-challenge-3/data/language_alignment_pt.jsonl'

dataset = load_dataset(
    "json",
    data_files="/content/drive/MyDrive/rag/language_alignment_pt.jsonl",  #Alterar quando o dataset já estiver na main
    split="train"
)

print(dataset.column_names)

# Define uma função para converter o formato de mensagens em texto para o treinamento do modelo.
def messages_to_text(example):
    text = ""
    for msg in example["messages"]:
        if msg["role"] == "system":
            text += f"<<SYS>>\n{msg['content']}\n<</SYS>>\n\n"
        elif msg["role"] == "user":
            text += f"[INST] {msg['content']} [/INST]\n"
        elif msg["role"] == "assistant":
            text += msg["content"]
    return {"text": text}

# Aplica a função de conversão ao dataset.
dataset = dataset.map(
    messages_to_text,
    batched=False,
    remove_columns=["messages"]
)

print(dataset.column_names)
print(dataset[0])

Cloning into 'tech-challenge-3'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 75 (delta 35), reused 9 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (75/75), 102.89 KiB | 1.41 MiB/s, done.
Resolving deltas: 100% (35/35), done.


Generating train split: 0 examples [00:00, ? examples/s]

['messages']


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

['text']
{'text': '<<SYS>>\nVocê é um assistente médico científico.\n<</SYS>>\n\n[INST] O que é hipertensão arterial? [/INST]\nCom base nas evidências disponíveis, a hipertensão arterial é uma condição crônica caracterizada pela elevação persistente da pressão arterial, estando associada a maior risco de eventos cardiovasculares.'}


# Configuração do Modelo para Treinamento com LoRA
## - Carrega modelo base e aplica adaptação LoRA para reduzir custo de treino.

In [10]:
# Carrega um modelo de linguagem pré-treinado (`llama-3-8b-bnb-4bit`) com configurações específicas de otimização.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
    device_map="auto"
)

# Aplica PEFT (Parameter-Efficient Fine-Tuning) com LoRA ao modelo para treinamento eficiente.
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 16,
    lora_dropout = 0.05,
)

==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2026.1.2 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


# Treinamento Supervisionado do Assistente Médico em Português

In [11]:
# Configura os argumentos para o treinamento do modelo, como diretório de saída, número de épocas, tamanho do lote e taxa de aprendizado.
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=4,
    per_device_train_batch_size=4,
    learning_rate=2e-4,
    logging_steps=5,
    save_strategy="epoch",
    fp16=True,
    report_to="none"
)

# Inicializa o SFTTrainer (Supervised Fine-tuning Trainer) com o modelo, tokenizer e dataset de treinamento.
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset, #DataSet com dados Em Português
    args=training_args,
    dataset_text_field="text",
    max_seq_length=512,
    packing=False
)

# Inicia o processo de treinamento do modelo.
trainer.train()

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20 | Num Epochs = 4 | Total steps = 20
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 13,631,488 of 8,043,892,736 (0.17% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
5,2.186400
10,1.927100
15,1.753900
20,1.663000


TrainOutput(global_step=20, training_loss=1.882569932937622, metrics={'train_runtime': 46.7452, 'train_samples_per_second': 1.711, 'train_steps_per_second': 0.428, 'total_flos': 338515472547840.0, 'train_loss': 1.882569932937622, 'epoch': 4.0})

# Autenticação e Upload do Modelo Treinado no HuggingFace

In [12]:
# Carrega variáveis de ambiente (como o token do Hugging Face) de um arquivo .env.
# Opicional: Subir modelo no Hugging Face
ENV_PATH = "/content/drive/MyDrive/token-hf/env"
load_dotenv(ENV_PATH)

# Realiza o login no Hugging Face Hub usando o token.
HF_TOKEN = os.getenv("HF_TOKEN")
login(token=HF_TOKEN)

# Define o nome do repositório no Hugging Face para o upload do modelo.
HF_REPO = f"{os.getenv("HF_USER_REPO")}/assistente-medico-lora"

# Realiza o upload do modelo treinado para o Hugging Face Hub.
model.push_to_hub(HF_REPO)
# Realiza o upload do tokenizer para o Hugging Face Hub.
tokenizer.push_to_hub(HF_REPO)

README.md:   0%|          | 0.00/550 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   1%|1         |  554kB / 54.6MB            

Saved model to https://huggingface.co/diegosdomingos/assistente-medico-lora


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mpo6rcqkk4/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

No files have been modified since last commit. Skipping to prevent empty commit.


# PROMPT utilizados no modelo

In [13]:
# Define o prompt do sistema para o assistente médico virtual, estabelecendo seu papel, regras e tom de voz.
SYSTEM_PROMPT = """
Você é um assistente médico virtual.
Responda sempre em português, com linguagem clara, empática e baseada em evidências científicas.

Regras:
- Não invente informações.
- Se não houver evidência suficiente, diga isso explicitamente.
- Não prescreva remédios ou medicamentos, e nem indique tratamentos específicos.
- Quando perguntarem por algum remédio, você deve responder: Não estou autorizado a prescrever medicamentos, por favor, consulte um médico. <FIM>
- Sempre cite a fonte da informação científica

Importante:
- Responda de forma resumida e objetiva e finalize sempre a primeira resposta objetiva com o texto: <FIM>
"""

# Pipeline utilizadas

In [14]:
# Configura o pipeline do LLM para consulta de artigos científicos, com baixa temperatura para respostas determinísticas.
llm_consulta = pipeline(
  "text-generation",
  model=model,
  tokenizer=tokenizer,
  max_new_tokens=250,
  temperature=0.0,
  do_sample=False,
  repetition_penalty=1.1,
  return_full_text=False,
  eos_token_id=tokenizer.eos_token_id
)

Device set to use cuda:0


# Busca informações no Q.A. pubMedQA

In [15]:
# Carrega os documentos médicos e o índice FAISS salvos anteriormente.
with open('/content/drive/MyDrive/rag/medical_docs.json') as f:
  docs = json.load(f)

index = faiss.read_index('/content/drive/MyDrive/rag/medical_index.faiss')

# Define uma função para recuperar contextos relevantes com base em uma pergunta, usando o índice FAISS.
def retrieve_context(question, k=3):
  q_emb = embedder.encode([question])
  _, idx = index.search(q_emb, k)
  return "\n\n".join([docs[i] for i in idx[0]])

# Define a função principal de chat médico que formula um prompt com contexto e gera uma resposta usando o LLM de consulta.
def query_QA(question):

  # Adiciona o token de fim à pergunta, se necessário.
  if "<FIM>" not in question:
    question += " <FIM>"

  # Recupera o contexto mais relevante para a pergunta.
  context = retrieve_context(question)

  # Constrói o prompt para o LLM, incluindo o prompt do sistema, o contexto e a pergunta.
  prompt = f"""
{SYSTEM_PROMPT}


Contexto científico relevante:
{context}


Pergunta: {question}
Resposta:
"""
  # Gera a resposta usando o LLM de consulta.
  output = llm_consulta(prompt)[0]["generated_text"]

  # Remove o token de fim e espaços em branco da resposta gerada.
  output = output.split("<FIM>")[0]

  return output.strip()

# Testes de Consulta ao Assistente Médico com Exemplos


In [16]:
# Exemplo de consulta normal ao assistente médico.
question = "O que a literatura indica sobre o uso de aspirina em prevenção primária?"
print(f"Resposta 1 (normal): {query_QA(question)}")

# Exemplo de consulta que testa a restrição do assistente em prescrever medicamentos.
question = "Qual medicamento é eficaz para pedra nos rins?"
print(f"Resposta 2 (restrição): {query_QA(question)}")

Resposta 1 (normal): A literatura indica que o uso de aspirina pode reduzir o risco de doença cardiovascular em pacientes sem história prévia de doenças cardiovasculares. No entanto, os benefícios do uso de aspirina devem ser avaliados contra seus riscos, incluindo sangramento gastrointestinal. Os pacientes que apresentam fatores de risco para doença cardiovascular, mas não têm histórico de doença cardiovascular, podem considerar o uso de aspirina como uma opção terapêutica.
Resposta 2 (restrição): Não estou autorizada a prescrever medicamentos, por favor, consulte um médico.


# Montagem das consultas utilizados para extrair prontuários do SQLite

In [17]:
# Função para buscar pacientes no banco de dados por nome parcial, considerando diferentes padrões.
def buscar_pacientes_por_nome(nome_parcial: str):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    bind = nome_parcial.strip()

    #print('[DEBUG BIND]', bind)

    # Executa a consulta SQL para encontrar pacientes com nomes correspondentes.
    cursor.execute("""
        SELECT patient_id, nome, idade
        FROM pacientes
        WHERE LOWER(nome) LIKE LOWER(?)
        OR LOWER(nome) LIKE LOWER(? || ' %')
        OR LOWER(nome) LIKE LOWER('% ' || ? || ' %')
        OR LOWER(nome) LIKE LOWER('% ' || ?)
    """, (bind, bind, bind, bind))

    resultados = cursor.fetchall()
    conn.close()
    #print('[DEBUG RESULT]', resultados)
    return resultados


# Função para buscar o prontuário de um paciente específico usando seu ID.
def buscar_prontuario_por_patient_id(patient_id: int):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    # Executa a consulta SQL para obter o diagnóstico e observações do atendimento.
    cursor.execute("""
        SELECT diagnostico, observacoes
        FROM atendimentos
        WHERE patient_id = ?
    """, (patient_id,))

    prontuario = cursor.fetchone()
    conn.close()
    return prontuario

# Controles para o chat do assistente


## - Utiliza o Spacy para extrair o nome do paciente



In [54]:
# Extrai o nome do paciente do texto
nlp = spacy.load("pt_core_news_lg")

def extrair_nome_paciente(texto: str):

    texto = texto.split()
    texto = " ".join(p.capitalize() for p in texto)

    doc = nlp(texto)
    nomes = [ent.text for ent in doc.ents if ent.label_ == "PER"]

    if not nomes:
        return None

    return nomes[0]


## - Cria um pequeno controle de estado para o CHAT

In [19]:
# Inicializa o estado do chat para controlar o modo (ex: prontuário) e dados parciais.
estado_chat = {
    "modo": None,            # None | "PRONTUARIO"
    "dados_parciais": {}     # ex: {"nome_paciente": "..."}
}

# Função para limpar o estado do chat, resetando o modo e os dados parciais.
def limpar_estado():
    estado_chat["modo"] = None
    estado_chat["dados_parciais"].clear()

## - Chama a consulta científica QA (RAG)

In [20]:
# Função para responder a perguntas científicas gerais utilizando o assistente médico.
def responder_qa(mensagem):
    resposta = query_QA(mensagem)

    if not resposta:
        return (
            "Não encontrei uma resposta científica clara no material disponível 📚\n\n"
            "Se quiser, pode reformular a pergunta."
        )

    return resposta

## - Chama a consulta ao prontuário (SQLite)

In [21]:
# Função para responder a consultas de prontuário, buscando informações do paciente no banco de dados.
def responder_prontuario_com_nome(nome: str):
    pacientes = buscar_pacientes_por_nome(nome)

    if not pacientes:
        limpar_estado()
        return f"Não encontrei nenhum prontuário para {nome}."

    if len(pacientes) > 1:
        lista = "\n".join(
            f"- {p[1]} (idade: {p[2]})"
            for p in pacientes
        )
        return (
            "Encontrei mais de um paciente com esse nome \n\n"
            "Por favor, confirme qual deles você deseja consultar:\n"
            f"{lista}"
        )

    patient_id, nome_completo, idade = pacientes[0]
    prontuario = buscar_prontuario_por_patient_id(patient_id)

    limpar_estado()

    diagnostico, observacoes = prontuario

    return (
        "Prontuário do paciente\n\n"
        f"• Nome: {nome_completo}\n"
        f"• Idade: {idade}\n"
        f"• Diagnóstico: {diagnostico}\n"
        f"• Observações: {observacoes}"
    )

## - Faz a análise do contexto da mensagem e classifica em 5 opções:

- PRONTUARIO - Consulta ao banco de prontuário (SQLite)
- PRECISA_MAIS_INFO - É uma consulta ao prontuário, mas faltam dados para completar a ação.
- QA - Dúvida científica, deve consultar o QA (RAG)
- INVALIDA - Textos vagos, sem sentido ou que não seja possível interpretar
- INDEVIDA - Solicitação indevida. Exemplo: Pedir que o chat receite um medicamento ou instruções de como administrá-lo.

In [52]:
zero_shot_classifier = pipeline(
    task="zero-shot-classification",
    model="joeddav/xlm-roberta-large-xnli",
    device=0 if torch.cuda.is_available() else -1
)

INTENT_MAP = {
    "PRONTUARIO": "consulta a prontuário médico de um paciente",
    "QA": "pergunta científica ou médica baseada em estudos",
    "INDEVIDA": "pedido de receita médica ou prescrição de medicamento",
    "INVALIDA": "mensagem inválida, sem sentido ou fora de contexto"
}

INTENT_LABELS = list(INTENT_MAP.values())

def classificar_intencao(texto: str):
    resultado = zero_shot_classifier(texto, INTENT_LABELS)

    label_natural = resultado["labels"][0]
    score = resultado["scores"][0]

    # Mapeamento reverso
    tag = None
    for k, v in INTENT_MAP.items():
        if v == label_natural:
            tag = k
            break

    if tag is None:
        return "INVALIDA"

    print("SCORE", score)

    # Lógica baseada em confiança
    if score >= 0.70:
        return tag

    return "INVALIDA"

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


# Respostas padrões para mensagens que não geram consulta à dados

In [23]:

RESPOSTAS_INVALIDA = [
    "Não consegui entender bem sua solicitação. Você pode reformular ou explicar um pouco melhor o que precisa?",
    "Sua mensagem não ficou muito clara para mim. Pode tentar explicar de outra forma?",
    "Tive dificuldade para compreender sua solicitação. Você poderia fornecer mais detalhes?",
    "Não entendi exatamente o que você deseja no momento. Pode esclarecer um pouco mais?",
    "Posso ajudar com consultas a prontuários específicos ou com perguntas científicas. Você pode reformular sua solicitação?",
    "Se você deseja consultar um prontuário, informe o nome do paciente. Caso contrário, explique melhor sua dúvida.",
    "Não consegui entender sua solicitação. Você pode informar se deseja consultar um prontuário ou fazer uma pergunta científica?",
    "Para que eu possa ajudar, indique o nome do paciente ou detalhe melhor o tipo de informação que você procura."
]

# Retorna uma mensagem padrão quando a entrada do usuário é inválida.
def resposta_invalida():
    return random.choice(RESPOSTAS_INVALIDA)


# Retorna uma mensagem padrão para solicitações indevidas (ex: prescrição de medicamentos).
def resposta_indevida():
    return (
        "Não tenho permissão para responder este tipo de dúvida  \n\n"
        "Somente um Médico está apto para receitar ou indicar medicamentos"
    )

def solicita_nome_paciente():
    return (
        "Por favor, informe o nome completo do paciente."
    )

# Router para decidir o que a llm deve consultar Q.A. científico ou Prontuário no SQLite

In [56]:
# Função principal do roteador de chat, que direciona a mensagem do usuário com base em sua classificação.
def chat_router(mensagem: str):
    global estado_chat

    # Se o chat estiver no modo "PRONTUARIO", tenta extrair o nome do paciente.
    if estado_chat["modo"] == "PRONTUARIO":
        nome = extrair_nome_paciente(mensagem)

        if not nome:
            return solicita_nome_paciente()

        limpar_estado()
        return responder_prontuario_com_nome(nome)

    # Classifica a mensagem do usuário para determinar o tipo de interação.
    classificacao = classificar_intencao(mensagem)

    print("Assunto: ",classificacao)

    # Se a mensagem for sobre um prontuário, tenta extrair o nome e responde.
    if classificacao == "PRONTUARIO":

        estado_chat["modo"] = "PRONTUARIO"

        nome = extrair_nome_paciente(mensagem)

        if nome:
            return responder_prontuario_com_nome(nome)

        # Se não houver nome, solicita mais informações e muda o modo do chat para "PRONTUARIO".
        return solicita_nome_paciente()

    # Se a mensagem for uma pergunta de QA, responde utilizando o LLM de consulta.
    if classificacao == "QA":
        limpar_estado()
        return responder_qa(mensagem)

    # Se a mensagem for indevida, retorna a resposta padrão para tal.
    if classificacao == "INDEVIDA":
        limpar_estado()
        return resposta_indevida()

    # Para qualquer outra classificação, retorna uma resposta de mensagem inválida.
    return resposta_invalida()


# Teste do assistente com a simulação de um CHAT real

In [57]:
# Inicia a interface de chat do assistente médico.
print("Assistente Médico iniciado")
print("Digite 'sair' para encerrar\n")

# Loop principal do chat, que continua até o usuário digitar 'sair'.
while True:
    user_input = input("Você: ").strip()

    # Verifica se o usuário deseja encerrar a sessão.
    if user_input.lower() in ["sair", "exit", "quit"]:
        print("Assistente: Sessão encerrada.")
        break

    # Roteia a mensagem do usuário para a função de resposta apropriada.
    resposta = chat_router(user_input)
    print(f"\nAssistente: {resposta}\n")
    print("\n")
    print("="*60)
    print("\n")


Assistente Médico iniciado
Digite 'sair' para encerrar

Você: consulte o prontuário médico de Ana
SCORE 0.9790962934494019
Assunto:  PRONTUARIO

Assistente: Por favor, informe o nome completo do paciente.





Você: ana paula

Assistente: Prontuário do paciente

• Nome: Ana Paula Souza
• Idade: 50
• Diagnóstico: Gastrite crônica
• Observações: Paciente orientado quanto aos sinais de alarme





Você: prontuário médico de Beatriz
SCORE 0.5217194557189941
Assunto:  INVALIDA

Assistente: Sua mensagem não ficou muito clara para mim. Pode tentar explicar de outra forma?





Você: prontuário médico de Ana
SCORE 0.9487354159355164
Assunto:  PRONTUARIO

Assistente: Por favor, informe o nome completo do paciente.





Você: ana paula

Assistente: Prontuário do paciente

• Nome: Ana Paula Souza
• Idade: 50
• Diagnóstico: Gastrite crônica
• Observações: Paciente orientado quanto aos sinais de alarme





Você: prontuário médico de Ana
SCORE 0.9487354159355164
Assunto:  PRONTUARIO

Assistente: Po

KeyboardInterrupt: Interrupted by user